In [ ]:
# Import RadarSimPy library and display version information
import radarsimpy

print("`RadarSimPy` used in this example is version: " + str(radarsimpy.__version__))

# Pulse-Doppler Radar: Long-Range Detection with Doppler Processing

## Introduction

This notebook demonstrates **Pulse-Doppler radar**—a sophisticated radar technique that combines pulsed transmission with coherent Doppler processing to achieve long-range detection while measuring target velocity. Pulse-Doppler is widely used in air surveillance, military applications, and weather radar where long range and velocity measurement are both critical.

### What is Pulse-Doppler Radar?

**Basic Concept:**

Pulse-Doppler radar transmits short **high-power pulses** and processes the **Doppler shift** across multiple pulses to extract both range and velocity information. It combines:
- **Pulsed radar**: High peak power for long range
- **Doppler processing**: Velocity measurement and clutter rejection

**Key Characteristics:**

1. **High peak power**: Enables long-range detection (hundreds of km)
2. **Low duty cycle**: Short pulses with long listening periods
3. **Coherent processing**: Phase-coherent across pulses
4. **Doppler filtering**: Separates targets by velocity
5. **MTI capability**: Moving Target Indication (clutter rejection)

### Pulse-Doppler Principles

**Range Measurement:**

Range is determined from the **time delay** of the pulse echo:

$$R = \frac{c \cdot \tau}{2}$$

Where:
- $R$ = Range (m)
- $c$ = Speed of light (3×10⁸ m/s)
- $\tau$ = Round-trip time delay (s)

**Range Resolution:**

The ability to separate two targets in range depends on pulse width:

$$\Delta R = \frac{c \cdot T_p}{2}$$

Where $T_p$ is the pulse duration.

**Maximum Unambiguous Range:**

Limited by the pulse repetition frequency (PRF):

$$R_{max} = \frac{c}{2 \cdot PRF} = \frac{c \cdot PRP}{2}$$

Where PRP = Pulse Repetition Period.

### Doppler Processing

**Doppler Shift:**

A moving target causes a frequency shift:

$$f_d = \frac{2v_r f_c}{c}$$

Where:
- $f_d$ = Doppler frequency (Hz)
- $v_r$ = Radial velocity (m/s)
- $f_c$ = Carrier frequency (Hz)

**Coherent Pulse Train:**

By transmitting multiple phase-coherent pulses and processing the returns, the radar creates a **Doppler filter bank** that separates targets by velocity.

**Velocity Resolution:**

$$\Delta v = \frac{c}{2f_c N \cdot PRP}$$

Where $N$ = Number of pulses.

### Pulse-Doppler Modes

**High PRF Mode:**
- PRF > 2 × Doppler frequency
- Long maximum velocity
- Range ambiguous (multiple trip returns)
- Good for fast-moving targets

**Medium PRF Mode:**
- Compromise between range and velocity
- Some range ambiguity
- Some velocity ambiguity
- Most versatile

**Low PRF Mode:**
- PRF << Doppler frequency
- Long unambiguous range
- Velocity ambiguous
- Traditional MTI radar

### Advantages of Pulse-Doppler

**1. Long Range:**
- High peak power (kW to MW)
- Low duty cycle allows cooling
- Pulse compression possible
- Ranges: 10-500 km typical

**2. Doppler Filtering:**
- Separate targets by velocity
- Reject stationary clutter (ground, rain)
- Detect moving targets in clutter
- Measure target velocity

**3. High Sensitivity:**
- Coherent integration across pulses
- Processing gain: 10×log₁₀(N) dB
- Effective clutter cancellation
- Low noise figure receivers

**4. Target Discrimination:**
- Velocity information aids identification
- Friend vs. foe determination
- Weather vs. aircraft separation
- Multiple target tracking

### Moving Target Indication (MTI)

**Clutter Problem:**

Ground clutter and weather returns can be millions of times stronger than aircraft echoes. Pulse-Doppler solves this through **Doppler filtering**.

**MTI Processing:**

- **Stationary clutter**: Zero Doppler (DC component)
- **Moving targets**: Non-zero Doppler
- **High-pass filter**: Removes DC, passes moving targets
- **Improvement factor**: 30-60 dB clutter rejection

### Pulse Compression

**Concept:**

Transmit a **long, coded pulse** for energy, then compress it to a **short pulse** for resolution:

$$\text{Range Resolution} = \frac{c}{2B}$$

Where $B$ is the bandwidth (independent of pulse length).

**Techniques:**
- Linear FM chirp (most common)
- Phase coding (Barker, polyphase)
- Frequency hopping

**Advantage:**

Long pulse (high energy, long range) with short pulse resolution.

### Applications

**Air Defense:**
- Fighter aircraft radar
- Surface-to-air missile guidance
- Air traffic control
- Airport surveillance

**Weather Radar:**
- Doppler weather radar
- Wind velocity measurement
- Storm tracking
- Aviation weather

**Space Surveillance:**
- Satellite tracking
- Space debris detection
- Ballistic missile warning
- Ionospheric research

**Military:**
- Airborne early warning (AWACS)
- Ground surveillance
- Naval fire control
- Missile seekers

### Waveform Parameters

**This Example:**

- **Carrier Frequency**: 10 GHz (X-band)
- **Maximum Range**: 5000 m (5 km)
- **Range Resolution**: 500 m
- **Pulse Width**: ~3.33 μs
- **PRF**: 30 kHz
- **PRP**: ~33.3 μs
- **Peak Power**: 5 MW (67 dBm)
- **Sampling Rate**: 6 MHz

### You'll Learn How To:

- Configure pulse-Doppler radar with proper waveform parameters
- Calculate pulse width from range resolution requirements
- Determine PRF from maximum unambiguous range
- Set up high-power pulsed transmission (67 dBm = 5 MW)
- Define rectangular pulse modulation envelope
- Configure directional antenna patterns
- Model complex 3D target using STL mesh (corner reflector)
- Simulate realistic radar returns with mesh density
- Extract Doppler phase from baseband signal
- Visualize phase evolution showing target motion
- Understand range-Doppler ambiguity relationships
- Appreciate pulse-Doppler advantages for long-range detection

### About This Example

This notebook uses [`RadarSimPy`](https://radarsimx.com/radarsimx/radarsimpy/) to demonstrate:

**Radar Configuration:**
- 10 GHz X-band pulse-Doppler radar
- 5 MW peak power (high-power pulsed)
- Directional antenna (cosine pattern)
- Single pulse for simplicity

**Target:**
- Corner reflector (STL 3D mesh model)
- Range: 2000 m
- Velocity: 150 m/s approaching
- Realistic scattering from mesh

**Processing:**
- Baseband I/Q signal capture
- Phase extraction for Doppler analysis
- Time-domain visualization

**Expected Results:**
- Phase progression showing Doppler shift
- Clean target return at 2000m range
- Demonstrates pulse-Doppler concept

The simulation provides insight into pulse-Doppler radar operation and the relationship between waveform parameters and radar performance.

## Radar System Configuration

Configure pulse-Doppler radar with appropriate waveform parameters for long-range detection.

### Import Required Modules

Import necessary modules for radar simulation and visualization.

In [ ]:
# Import necessary modules for radar simulation
from radarsimpy import Radar, Receiver, Transmitter
from radarsimpy.simulator import sim_radar
from scipy import constants
import numpy as np
import plotly.graph_objs as go
from IPython.display import Image

### Antenna Pattern Definition

Define directional antenna radiation patterns for transmit and receive.

**Antenna Characteristics:**

- **Gain**: 20 dBi (high-gain directional antenna)
- **Azimuth Pattern**: cos⁵⁰⁰ taper (very narrow beam)
- **Elevation Pattern**: cos⁴⁰⁰ taper (narrow beam)
- **Angular Coverage**: ±20° (pencil beam)

**Purpose:**

High-gain antennas provide:
1. **Long range**: More power on target
2. **Directionality**: Know target azimuth/elevation
3. **Low sidelobes**: Reduced clutter pickup
4. **Spatial filtering**: Angular discrimination

In [ ]:
# Define antenna gain and angular coverage
antenna_gain = 20  # dBi - High-gain directional antenna

# Speed of light constant
light_speed = constants.c  # 299,792,458 m/s

### Azimuth Pattern (Horizontal Plane) ###

# Define azimuth angle array: -20° to +20° in 1° steps
az_angle = np.arange(-20, 21, 1)  # 41 points

# Azimuth pattern: cos^500 taper with 20 dBi gain
# Very narrow beamwidth (~2° @ -3dB) with low sidelobes
az_pattern = 20 * np.log10(np.cos(az_angle / 180 * np.pi) ** 500) + antenna_gain  # dB

### Elevation Pattern (Vertical Plane) ###

# Define elevation angle array: -20° to +20° in 1° steps
el_angle = np.arange(-20, 21, 1)  # 41 points

# Elevation pattern: cos^400 taper with 20 dBi gain
# Narrow beamwidth (~2.5° @ -3dB)
el_pattern = 20 * np.log10((np.cos(el_angle / 180 * np.pi)) ** 400) + antenna_gain  # dB

### Waveform Parameter Calculation

Calculate pulse-Doppler waveform parameters based on performance requirements.

**Design Requirements:**

1. **Maximum Unambiguous Range**: 5000 m (5 km)
2. **Range Resolution**: 500 m

**Parameter Derivation:**

From these requirements, we calculate:

**1. Pulse Bandwidth:**

$$B = \frac{c}{2 \Delta R} = \frac{3 \times 10^8}{2 \times 500} = 300 \text{ kHz}$$

**2. Pulse Width:**

$$T_p = \frac{1}{B} = \frac{1}{300,000} \approx 3.33 \text{ μs}$$

**3. Pulse Repetition Frequency (PRF):**

$$PRF = \frac{c}{2R_{max}} = \frac{3 \times 10^8}{2 \times 5000} = 30 \text{ kHz}$$

**4. Pulse Repetition Period (PRP):**

$$PRP = \frac{1}{PRF} = \frac{1}{30,000} \approx 33.3 \text{ μs}$$

**5. Duty Cycle:**

$$\text{Duty Cycle} = \frac{T_p}{PRP} = \frac{3.33}{33.3} = 10\%$$

This is a typical low duty cycle for pulsed radar, allowing high peak power with reasonable average power.

In [ ]:
### Calculate Pulse-Doppler Waveform Parameters ###

# Performance Requirements
max_range = 5000        # Maximum unambiguous range: 5000 m (5 km)
range_res = 500         # Required range resolution: 500 m

# Derived Waveform Parameters
pulse_bw = light_speed / (2 * range_res)  # Pulse bandwidth: c/(2*ΔR) = 300 kHz
pulse_width = 1 / pulse_bw                # Pulse width: 1/B ≈ 3.33 μs
prf = light_speed / (2 * max_range)       # PRF: c/(2*R_max) = 30 kHz
prp = 1 / prf                             # PRP: 1/PRF ≈ 33.3 μs

# Radar Parameters
fs = 6e6                # Sampling rate: 6 MHz (> 2 × pulse_bw for Nyquist)
fc = 10e9               # Carrier frequency: 10 GHz (X-band)
num_pulse = 1           # Number of pulses: 1 (single pulse for this example)

# Calculate sample counts
total_samples = int(prp * num_pulse * fs)    # Total samples in observation
sample_per_pulse = int(total_samples / num_pulse)  # Samples per pulse

print(f"Pulse-Doppler Waveform Parameters:")
print(f"  Pulse Width: {pulse_width*1e6:.2f} μs")
print(f"  Pulse Bandwidth: {pulse_bw/1e3:.1f} kHz")
print(f"  PRF: {prf/1e3:.1f} kHz")
print(f"  PRP: {prp*1e6:.2f} μs")
print(f"  Duty Cycle: {(pulse_width/prp)*100:.1f}%")
print(f"  Samples per Pulse: {sample_per_pulse}")

### Pulse Envelope Generation

Create rectangular pulse modulation envelope for the transmitter.

**Pulse Modulation:**

The amplitude modulation array defines the pulse shape:
- **Pulse-on** (0 to pulse_width): Amplitude = 1
- **Pulse-off** (after pulse_width): Amplitude = 0

**Time Array:**

The modulation time array `mod_t` spans the entire PRP with sampling at `fs`, defining when the pulse is transmitted within each repetition period.

In [ ]:
### Create Rectangular Pulse Envelope ###

# Create time array for modulation (spans entire PRP)
mod_t = np.arange(0, total_samples) / fs  # Time vector (seconds)

# Initialize amplitude modulation array (all zeros initially)
amp = np.zeros_like(mod_t)

# Set pulse-on period: amplitude = 1 for duration of pulse_width
amp[mod_t <= pulse_width] = 1  # Rectangular pulse

# Result: Pulse is ON for first ~3.33 μs, then OFF for remaining ~30 μs

### Transmitter Configuration

Configure high-power pulsed transmitter with directional antenna pattern.

**Transmitter Parameters:**

- **Carrier Frequency**: 10 GHz (X-band)
- **Pulse Duration**: 1/PRF ≈ 33.3 μs (PRP)
- **Transmit Power**: 67 dBm = 5 MW peak power
  - High peak power typical for long-range pulse-Doppler
  - Average power: 5 MW × 10% duty = 500 kW
- **Number of Pulses**: 1 (single pulse for demonstration)
- **Modulation**: Rectangular pulse envelope (amp, mod_t)
- **Antenna Pattern**: Directional with 20 dBi gain

**High Peak Power:**

Pulse-Doppler radars use very high peak power (MW range) to achieve long detection ranges, but the low duty cycle keeps average power manageable.

In [ ]:
# Define transmitter channel with antenna pattern and pulse modulation
tx_channel = dict(
    location=(0, 0, 0),              # Antenna position at origin
    azimuth_angle=az_angle,          # Azimuth angles array
    azimuth_pattern=az_pattern,      # Azimuth gain pattern (dB)
    elevation_angle=el_angle,        # Elevation angles array
    elevation_pattern=el_pattern,    # Elevation gain pattern (dB)
    amp=amp,                         # Pulse amplitude modulation
    mod_t=mod_t,                     # Modulation time array
)

# Create pulse-Doppler transmitter
tx = Transmitter(
    f=fc,                # Carrier frequency: 10 GHz
    t=1 / prf,           # Pulse duration: PRP ≈ 33.3 μs
    tx_power=67,         # Transmit power: 67 dBm = 5 MW (very high peak power!)
    pulses=num_pulse,    # Number of pulses: 1
    channels=[tx_channel],  # Transmitter antenna configuration
)

### Receiver Configuration

Configure receiver for baseband sampling and signal conditioning.

**Receiver Parameters:**

- **Sampling Rate**: 6 MHz
  - Exceeds Nyquist for 300 kHz bandwidth
  - Captures pulse envelope clearly
  
- **Noise Figure**: 12 dB (standard performance)

- **RF Gain**: 20 dB (LNA amplification)

- **Baseband Gain**: 30 dB (total gain: 50 dB)

- **Load Resistor**: 500 Ω

- **Antenna Pattern**: Same directional pattern as transmitter (monostatic)

**Monostatic Configuration:**

TX and RX share the same location and antenna pattern, typical for most radar systems.

In [ ]:
# Define receiver channel with same antenna pattern
rx_channel = dict(
    location=(0, 0, 0),              # Receiver position at origin (monostatic)
    azimuth_angle=az_angle,          # Same azimuth pattern as TX
    azimuth_pattern=az_pattern,      # Same azimuth gain
    elevation_angle=el_angle,        # Same elevation pattern
    elevation_pattern=el_pattern,    # Same elevation gain
)

# Create pulse-Doppler receiver
rx = Receiver(
    fs=fs,               # Sampling rate: 6 MHz
    noise_figure=12,     # Noise figure: 12 dB
    rf_gain=20,          # RF gain: 20 dB
    load_resistor=500,   # Load resistance: 500 Ω
    baseband_gain=30,    # Baseband gain: 30 dB
    channels=[rx_channel],  # Receiver antenna configuration
)

### Create Radar System

Combine transmitter and receiver to form the complete pulse-Doppler radar.

In [ ]:
# Create complete pulse-Doppler radar system
radar = Radar(transmitter=tx, receiver=rx)

## Target Configuration

Define a realistic 3D target using STL mesh model for accurate scattering simulation.

### 3D Mesh Target

**Corner Reflector Model:**

Instead of a simple point target, this example uses a **corner reflector** defined by an STL (STereoLithography) mesh file. This provides:

1. **Realistic Scattering**: Accurate radar cross-section based on geometry
2. **Angular Dependence**: RCS varies with aspect angle
3. **Multiple Scattering**: Interactions between surfaces
4. **Physical Accuracy**: Real-world target representation

**Target Parameters:**

- **Model**: `cr.stl` (corner reflector geometry)
- **Units**: meters
- **Location**: (2000, 0, 0) m → 2 km range
- **Velocity**: (150, 0, 0) m/s → 150 m/s approaching (540 km/h)
  - Very high velocity (supersonic: Mach 0.44)
  - Creates significant Doppler shift
  
**Mesh Density:**

The `density=0.5` parameter controls mesh sampling:
- Higher density: More accurate but slower
- Lower density: Faster but less accurate
- 0.5 provides good balance

**High Velocity:**

150 m/s creates large Doppler shift:

$$f_d = \frac{2 \times 150 \times 10 \times 10^9}{3 \times 10^8} = 10 \text{ kHz}$$

This Doppler shift will be visible in the phase progression.

In [ ]:
# Define 3D mesh target (corner reflector)
target = {
    "model": "../models/cr.stl",  # Path to STL mesh file (corner reflector)
    "unit": "m",                   # Model units: meters
    "location": (2000, 0, 0),      # Position: 2000m range, on-axis
    "speed": (150, 0, 0),          # Velocity: 150 m/s approaching (very fast!)
}

# Alternative: Simple point target (commented out)
# target = dict(location=(1000, 0, 0), speed=(150, 0, 0), rcs=10, phase=0)

## Simulate Baseband Signals

Generate pulse-Doppler radar baseband signals with realistic 3D target scattering.

### Simulation Process

The simulator performs:
1. **Mesh Loading**: Read STL file and sample surface points
2. **Ray Tracing**: Calculate propagation paths to/from each mesh point
3. **Scattering**: Compute reflected signal from each surface element
4. **Doppler**: Apply frequency shift from target motion
5. **Integration**: Sum contributions from all mesh points
6. **Baseband**: Down-convert to complex I/Q signal

**Simulation Parameters:**

- `density=0.5`: Mesh sampling density (balance accuracy vs. speed)
- `debug=False`: Suppress debug output for clean execution

**Output:**

- **timestamp**: Time array for each sample [1, 1, samples]
- **baseband**: Complex I/Q signal [1, 1, samples]
  - Note: Noise intentionally omitted for clear Doppler visualization

**No Noise:**

For this demonstration, receiver noise is NOT added to clearly show the Doppler phase progression. In practice, noise would be present.

In [ ]:
# Simulate pulse-Doppler radar with 3D mesh target
data = sim_radar(
    radar,           # Pulse-Doppler radar configuration
    [target],        # List of targets (corner reflector mesh)
    density=0.5,     # Mesh sampling density (0.5 = medium)
    debug=False      # Suppress debug output
)

# Extract simulation results
timestamp = data["timestamp"]  # Time array [1, 1, samples]
baseband = data["baseband"]    # Complex I/Q (noise omitted intentionally) [1, 1, samples]

print(f"Simulation complete:")
print(f"  Baseband shape: {baseband.shape}")
print(f"  Duration: {timestamp[0, 0, -1]*1e6:.2f} μs")
print(f"  Samples: {len(timestamp[0, 0, :])}")

## Signal Analysis

Extract and visualize Doppler information from the pulse-Doppler return.

### Phase Extraction

**Doppler Phase:**

For a moving target, the phase of the baseband signal evolves linearly with time:

$$\phi(t) = 2\pi f_d t = \frac{4\pi v_r f_c}{c} t$$

Where:
- $\phi(t)$ = Phase (radians)
- $f_d$ = Doppler frequency (Hz)
- $v_r$ = Radial velocity (m/s)
- $f_c$ = Carrier frequency (Hz)

**For This Example:**

- Velocity: $v_r = 150$ m/s (approaching, so negative Doppler)
- Carrier: $f_c = 10$ GHz
- Doppler frequency: $f_d = \frac{2 \times 150 \times 10^{10}}{3 \times 10^8} = 10$ kHz

**Phase Evolution:**

Over the ~33 μs observation:
- Phase change: $\Delta\phi = 2\pi \times 10,000 \times 33 \times 10^{-6} \approx 2.07$ radians ≈ 119°

**Extraction Method:**

Phase is extracted using `arctan2(I, Q)` which properly handles all quadrants.

### Visualize Phase Evolution

Display phase progression showing Doppler shift from target motion.

In [ ]:
# Extract phase from complex baseband signal
# arctan2(I, Q) provides phase in range [-π, +π]
phase = np.arctan2(np.real(baseband[0, 0, :]), np.imag(baseband[0, 0, :]))

# Create figure for phase visualization
fig = go.Figure()

# Plot phase vs. time
fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :] * 1e6,  # Convert to microseconds
        y=phase,                      # Phase in radians
        name="Doppler Phase",
        line=dict(color='purple', width=2),
        mode='lines',
    )
)

# Configure plot layout
fig.update_layout(
    title="Phase Evolution: Doppler Shift from 150 m/s Target Motion",
    yaxis=dict(title="Phase (radians)", gridcolor='lightgray'),
    xaxis=dict(title="Time (μs)", gridcolor='lightgray'),
    height=500,
)

# Add annotation explaining Doppler
fig.add_annotation(
    text=f"Doppler frequency ≈ 10 kHz<br>Phase slope ∝ target velocity",
    xref="paper", yref="paper",
    x=0.05, y=0.95,
    showarrow=False,
    bgcolor="rgba(255,255,200,0.8)",
    bordercolor="purple",
    borderwidth=2,
)

# uncomment this to display interactive plot
# fig.show()

# display static image to reduce size on radarsimx.com
img_bytes = fig.to_image(format="jpg", scale=2)
display(Image(img_bytes))

### Visualize In-Phase Component

Display the in-phase (I) component of the baseband signal showing the pulse return.

In [ ]:
# Create figure for in-phase (I) component
fig = go.Figure()

# Plot I component vs. time
fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :] * 1e6,      # Convert to microseconds
        y=np.real(baseband[0, 0, :]),    # Real part (I component)
        name="I (In-phase)",
        line=dict(color='blue', width=2),
        mode='lines',
    )
)

# Configure plot layout
fig.update_layout(
    title="Baseband In-Phase (I) Component: Pulse Return from Target",
    yaxis=dict(title="Amplitude (V)", gridcolor='lightgray'),
    xaxis=dict(title="Time (μs)", gridcolor='lightgray'),
    template="plotly_dark",
    height=500,
)

# uncomment this to display interactive plot
# fig.show()

# display static image to reduce size on radarsimx.com
img_bytes = fig.to_image(format="jpg", scale=2)
display(Image(img_bytes))

## Summary

In this notebook, you learned how to implement and analyze pulse-Doppler radar using RadarSimPy:

### Key Takeaways

**1. Pulse-Doppler Principles**
- Understood high-power pulsed transmission for long range
- Learned Doppler processing for velocity measurement
- Recognized range-time delay relationship: R = cτ/2
- Appreciated coherent integration advantages

**2. Waveform Design**
- Calculated pulse width from range resolution: Tp = c/(2ΔR)
- Determined PRF from maximum range: PRF = c/(2Rmax)
- Understood duty cycle: ~10% typical for pulsed radar
- Balanced range and resolution requirements

**3. High Peak Power**
- Configured 67 dBm (5 MW) transmit power
- Understood peak vs. average power trade-off
- Recognized long-range capability from high power
- Appreciated cooling benefits of low duty cycle

**4. Directional Antenna**
- Defined narrow pencil beam patterns (cos⁵⁰⁰, cos⁴⁰⁰)
- Achieved 20 dBi gain for long range
- Understood spatial filtering benefits
- Configured monostatic geometry

**5. 3D Mesh Target**
- Modeled realistic corner reflector using STL mesh
- Simulated accurate radar cross-section
- Captured angular scattering effects
- Achieved physical accuracy beyond point targets

**6. Doppler Detection**
- Extracted phase from complex I/Q signal
- Observed linear phase progression from velocity
- Calculated 10 kHz Doppler from 150 m/s motion
- Validated Doppler formula: fd = 2vfₒ/c

**7. Range-Doppler Ambiguity**
- Understood maximum unambiguous range: Rmax = c/(2×PRF)
- Recognized PRF selection trade-offs
- Appreciated high/medium/low PRF modes
- Understood velocity ambiguity considerations

**8. Pulse-Doppler Advantages**
- Long range from high peak power (hundreds of km)
- Doppler filtering for clutter rejection (MTI)
- Velocity measurement capability
- Target discrimination through Doppler

### Experiment Further

Try modifying the parameters to explore different scenarios:

**1. Range Requirements**:
- Vary max_range: 10 km, 50 km, 200 km
- Observe PRF changes
- Test range-velocity ambiguity
- Understand PRF selection

**2. Range Resolution**:
- Change range_res: 100m, 1000m, 5000m
- Observe pulse width changes
- Test bandwidth-resolution relationship
- Balance resolution vs. energy

**3. Target Velocity**:
- Vary speed: 50 m/s, 300 m/s, 500 m/s
- Observe Doppler frequency changes
- Test velocity ambiguity limits
- Validate Doppler formula

**4. Target Range**:
- Place target at 500m, 1000m, 4000m
- Observe return delay
- Test maximum range limits
- Understand propagation loss

**5. Carrier Frequency**:
- Test 3 GHz, 35 GHz, 77 GHz
- Observe Doppler sensitivity: fd ∝ fₒ
- Compare wavelength effects
- Understand frequency selection

**6. Peak Power**:
- Vary tx_power: 50 dBm, 70 dBm, 80 dBm
- Observe detection range changes
- Test SNR improvements
- Understand power-range relationship

**7. PRF Modes**:
- High PRF: 100 kHz (range ambiguous)
- Medium PRF: 30 kHz (current)
- Low PRF: 1 kHz (velocity ambiguous)
- Compare mode trade-offs

**8. Number of Pulses**:
- Increase to 32, 64, 128 pulses
- Perform Doppler FFT across pulses
- Observe coherent integration gain
- Generate range-Doppler map

**9. Antenna Beamwidth**:
- Wider beam: cos¹⁰⁰ (reduce exponent)
- Narrower beam: cos¹⁰⁰⁰ (increase exponent)
- Observe gain-beamwidth trade-off
- Test coverage vs. resolution

**10. Pulse Compression**:
- Implement linear FM chirp
- Add phase coding
- Test compression gain
- Achieve long pulse with short resolution

### Resources

- RadarSimPy Documentation: https://radarsimx.com
- RadarSimPy Simulator Module: https://radarsimx.github.io/radarsimpy/simulator.html
- Pulse-Doppler Radar: Pulsed waveform with Doppler processing
- Doppler Effect: Frequency shift from relative motion
- MTI Radar: Moving Target Indication and clutter rejection
- PRF Selection: Range-velocity ambiguity trade-offs
- Pulse Compression: Achieving range resolution with long pulses
- 3D Target Modeling: STL mesh for realistic scattering
- Air Surveillance: Long-range detection applications
- Coherent Integration: Processing gain from multiple pulses